In [1]:
!pip install flask


In [ ]:
from flask import Flask, render_template_string, request
import speech_recognition as sr
from googletrans import Translator

app = Flask(__name__)
translator = Translator()
recognizer = sr.Recognizer()

# HTML template for the chatbot interface
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Real-Time Language Translater </title>
    <script>
        function startRecording() {
            // Use the Web Speech API to capture audio
            var recognition = new webkitSpeechRecognition(); // For Chrome
            recognition.continuous = false;
            recognition.interimResults = false;

            recognition.onstart = function() {
                console.log("Voice recognition started. Try speaking into the microphone.");
            };

            recognition.onresult = function(event) {
                var transcript = event.results[0][0].transcript;
                document.getElementById("text_input").value = transcript; // Set input field with speech text
                document.getElementById("translate_form").submit(); // Automatically submit the form
            };

            recognition.onerror = function(event) {
                console.error("Speech recognition error", event);
            };

            recognition.start(); // Start voice recognition
        }
    </script>
</head>
<body>
    <h1>Real-Time language Translation</h1>
    <form method="post" id="translate_form">
        <label>Speak to translate:</label>
        <input type="text" id="text_input" name="text_input" placeholder="Your speech will appear here..."><br><br>
        
        <button type="button" onclick="startRecording()">Start Speaking</button><br><br>

        <label>Choose target language:</label>
        <select name="target_language">
            <option value="es">Spanish</option>
            <option value="hi">Hindi</option>
            <option value="fr">French</option>
            <option value="de">German</option>
            <option value="zh-cn">Chinese</option>
            <option valu="eng">English</option>
        </select><br><br>

        <input type="submit" value="Translate">
    </form>

    {% if translated_text %}
        <h2>Translated Text:</h2>
        <p>{{ translated_text }}</p>
    {% endif %}
</body>
</html>
"""

# Function to handle translation
def translate_text(text, target_language):
    try:
        translation = translator.translate(text, dest=target_language)
        return translation.text
    except Exception as e:
        print(f"Translation error: {e}")
        return None

# Route for chatbot interface
@app.route("/", methods=["GET", "POST"])
def chatbot():
    translated_text = None

    if request.method == "POST":
        text = request.form.get("text_input", "")
        target_language = request.form["target_language"]

        # Translate text if available
        if text:
            translated_text = translate_text(text, target_language)

    return render_template_string(html_template, translated_text=translated_text)

# Run the app
if __name__ == "__main__":
    app.run(port=5000)
